In [6]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import models, layers
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [7]:
import os
import shutil
from sklearn.model_selection import train_test_split

# Ursprünglicher Pfad zum Dataset
original_dataset_path = 'images/archive/bloodcells_dataset'

# Zielverzeichnisse für Trainings- und Validierungsdaten erstellen
train_dir = 'images/archive/bloodcells_dataset/train'
valid_dir = 'images/archive/bloodcells_dataset/validation'

# Erstelle die Verzeichnisse, falls sie nicht existieren
os.makedirs(train_dir, exist_ok=True)
os.makedirs(valid_dir, exist_ok=True)

# Liste der Klassen
classes = ['basophil', 'eosinophil', 'erythroblast', 'ig', 'lymphocyte', 'monocyte', 'neutrophil', 'platelet']

# Schleife durch jede Klasse und teile die Bilder auf
for cls in classes:
    # Ursprünglicher Pfad zu den Bildern einer Klasse
    class_path = os.path.join(original_dataset_path, cls)

    # Zielverzeichnisse für Trainings- und Validierungsdaten dieser Klasse erstellen
    train_class_dir = os.path.join(train_dir, cls)
    valid_class_dir = os.path.join(valid_dir, cls)

    os.makedirs(train_class_dir, exist_ok=True)
    os.makedirs(valid_class_dir, exist_ok=True)

    # Liste aller Bilder in der aktuellen Klasse
    images = os.listdir(class_path)

    # Aufteilung in Trainings- und Validierungssätze
    train_images, valid_images = train_test_split(images, test_size=0.2, random_state=42)

    # Kopiere Bilder in die entsprechenden Verzeichnisse
    for img in train_images:
        src_path = os.path.join(class_path, img)
        dest_path = os.path.join(train_class_dir, img)
        shutil.copy(src_path, dest_path)

    for img in valid_images:
        src_path = os.path.join(class_path, img)
        dest_path = os.path.join(valid_class_dir, img)
        shutil.copy(src_path, dest_path)

# Nun kannst du den vorherigen Transfer Learning-Code verwenden
# und dabei die aktualisierten Pfade für Trainings- und Validierungsdaten verwenden.


In [8]:
# Definiere die Pfade zu den Trainings- und Validierungsdaten
train_dir = 'images/archive/bloodcells_dataset/train'
valid_dir = 'images/archive/bloodcells_dataset/validation'

In [9]:
# Schritte 1 und 2: Modell auswählen und anpassen
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))


In [10]:
model = models.Sequential()
model.add(base_model)
model.add(layers.GlobalAveragePooling2D())
model.add(layers.Dense(8, activation='softmax'))

In [11]:
# Schritt 3: Daten vorbereiten und Generatoren erstellen
train_datagen = ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
)

valid_datagen = ImageDataGenerator(rescale=1./255)

img_size = (224, 224)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

valid_generator = valid_datagen.flow_from_directory(
    valid_dir,
    target_size=img_size,
    batch_size=32,
    class_mode='categorical'
)

Found 13671 images belonging to 8 classes.
Found 3421 images belonging to 8 classes.


In [12]:
# Schritt 4: Modell kompilieren
model.compile(optimizer=Adam(lr=0.001), loss='categorical_crossentropy', metrics=['accuracy'])


/Users/retosutter/opt/anaconda3/envs/DS2/lib/python3.8/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [ ]:
# Schritt 5: Modell trainieren
model.fit(train_generator, epochs=10, validation_data=valid_generator)

Epoch 1/10
  4/428 [..............................] - ETA: 2:33:51 - loss: 1.4599 - accuracy: 0.5391